In [ ]:
# edit the part after /content/drive/ with the Google Drive directories you want to use for images, inferences and classifier model
imageDirectory = "/content/drive/pathToWhereImagesAre/"
inferencesDirectory = "/content/drive/pathToWhereYouWantInferencesToGo/"
modelDirectory = "/content/drive/pathToModel/6:30:22 egg model/model.savedmodel"

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
!pip install requests_toolbelt
import io
import os
import cv2
import copy
import shutil
import requests
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image, ImageOps
from requests_toolbelt.multipart.encoder import MultipartEncoder
tf.keras.backend.manual_variable_initialization(True)

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def classifyObject(cv2Image):
    image = Image.fromarray(np.uint8(cv2Image)).convert('RGB')
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    size = (224, 224)
    # image = ImageOps.fit(image, size, Image.NEAREST)
    image = ImageOps.fit(image, size, Image.LANCZOS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
    data[0] = normalized_image_array
    prediction = model.predict(data, verbose = False)
    return(prediction)

def getBorders(cv2Image):
    src = cv2.cvtColor(cv2Image, cv2.COLOR_BGR2GRAY)
    dst = cv2.Canny(src, 10, 30, None, 3)
    column1 = np.sum(dst[:,range(0, int(dst.shape[1]/2))], axis = 0).argmax()
    column2 = int(dst.shape[1]/2) + np.sum(dst[:,range(int(dst.shape[1]/2),dst.shape[1])], axis = 0).argmax()
    row1 = np.sum(dst[range(0, int(dst.shape[0]/2)),:], axis = 1).argmax()
    row2 = int(dst.shape[0]/2) + np.sum(dst[range(int(dst.shape[0]/2),dst.shape[0]),:], axis = 1).argmax()
    if(abs((column2 - column1) - 4416) > 100):
      print("col corrected")
      column1 = 227
      column2 = 4643
    if(abs((row2 - row1) - 2196) > 75):
      print("row corrected")
      row1 = 725
      row2 = 2979
    return([row1, row2, column1, column2])

In [ ]:
targetSize = (3416, 2604)
minimumConfidence = 0.0
xBoxMargin = 50
yBoxMargin = 50

model = tf.keras.models.load_model(modelDirectory)

In [ ]:
os.chdir(inferencesDirectory)
images = list(map(lambda x : imageDirectory + x, os.listdir(imageDirectory)))
if("inferencesInProgress.csv" in os.listdir()):
  inferences = pd.read_csv(inferencesDirectory + "inferencesInProgress.csv")
else:
  inferences = pd.DataFrame(columns = ['imageCode', 'width', 'height', 'class', 'inframe', 'xmin', 'ymin', 'xmax', 'ymax'])
print(inferences)

                                 imageCode  width  height    class  inframe  \
0        22MIGS2HFL_0201_GS20-05-07_3_1314   3416    2604      egg        1   
1        22MIGS2HFL_0201_GS20-05-07_3_1314   3416    2604      egg        0   
2        22MIGS2HFL_0201_GS20-05-07_3_1316   3416    2604      egg        0   
3        22MIGS2HFL_0057_GS20-21-09_1_1319   3416    2604      egg        1   
4        22MIGS2HFL_0057_GS20-21-09_1_1320   3416    2604      egg        0   
...                                    ...    ...     ...      ...      ...   
3225     22MIGS2HFL_0077_GS20-31-09_1_0180   3416    2604  not egg        0   
3226     22MIGS2HFL_0077_GS20-31-09_1_0183   3416    2604      egg        1   
3227  22MIGS2HFL_0282_Covington_Extra_0186   3416    2604  not egg        0   
3228  22MIGS2HFL_0282_Covington_Extra_0188   3416    2604      egg        1   
3229  22MIGS2HFL_0282_Covington_Extra_0191   3416    2604      egg        0   

      xmin  ymin  xmax  ymax  
0     2746  1560  28

In [ ]:
# using improved roboflow inference filtering
with tf.device('/device:GPU:0'):
  for imageNum, imagePath in enumerate(images):
      thisImageCode = str(os.path.splitext(os.path.basename(imagePath))[0])
      if(thisImageCode in list(inferences['imageCode'])):
        continue

      print(str(imageNum) + " out of " + str(len(images)) + " images")
      print(thisImageCode)

      thisImage = cv2.imread(imagePath)
      thisBorders = getBorders(thisImage)

      thisBorders = [
        thisBorders[0] * (targetSize[1] / thisImage.shape[0]),
        thisBorders[1] * (targetSize[1] / thisImage.shape[0]),
        thisBorders[2] * (targetSize[0] / thisImage.shape[1]),
        thisBorders[3] * (targetSize[0] / thisImage.shape[1])
      ]
      thisBorders = [int(i) for i in thisBorders]

      thisImage = cv2.resize(thisImage, targetSize)
      thisCroppedImagePath = str(os.path.splitext(os.path.basename(imagePath))[0]) + "_cropped.jpg"
      cv2.imwrite(thisCroppedImagePath, thisImage)

      annotatedImage = copy.deepcopy(thisImage)
      cv2.rectangle(annotatedImage, (thisBorders[2], thisBorders[0]), (thisBorders[3], thisBorders[1]), 1, 5)
      greyscaleImage = copy.deepcopy(thisImage)
      greyscaleImage = cv2.cvtColor(greyscaleImage, cv2.COLOR_BGR2GRAY)
      greyscaleImage = cv2.merge([greyscaleImage,greyscaleImage,greyscaleImage])
      thisImage = copy.deepcopy(greyscaleImage)
      pilImage = Image.fromarray(greyscaleImage)
      buffered = io.BytesIO()
      pilImage.save(buffered, quality=100, format="JPEG")
      m = MultipartEncoder(fields={'file': ("imageToUpload", buffered.getvalue(), "image/jpeg")})
      response = requests.post("https://detect.roboflow.com/nematodeeggs3/1?api_key=JkhApD7IiYz1ti7jV6cy", data=m, headers={'Content-Type': m.content_type})
      # response = rfmodel.predict(thisCroppedImagePath, confidence = 1, overlap = 40)
      objects = response.json()["predictions"]

      count = 0
      for i in range(0, len(objects)):
          thisObject = objects[i]
          if(thisObject["confidence"] >= minimumConfidence):
              thisCoordinates = [
                  int(thisObject["y"] - yBoxMargin),
                  int((thisObject["y"] + thisObject["height"]) - yBoxMargin),
                  int(thisObject["x"] - xBoxMargin),
                  int(thisObject["x"] + thisObject["width"] - xBoxMargin)
              ]
              thisCentroid = [(thisCoordinates[0] + thisCoordinates[1]) / 2, (thisCoordinates[2] + thisCoordinates[3]) / 2]
              objectIsInFrame = (
                  thisBorders[0] < thisCentroid[0] < thisBorders[1] and
                  thisBorders[2] < thisCentroid[1] < thisBorders[3]
              )
              try:
                  thisThisImage = thisImage[range(thisCoordinates[0], thisCoordinates[1]),:]
                  thisThisImage = thisThisImage[:,range(thisCoordinates[2], thisCoordinates[3])]
              except:
                  continue

              thisPath = thisImageCode + "/" + str(i) + ".jpg"
              modelClasses = ("egg", "not egg", "double egg")
              prediction = classifyObject(thisThisImage)
              prediction = modelClasses[np.argmax(prediction)]

              inferences.loc[inferences.shape[0]] = [thisImageCode, thisImage.shape[1], thisImage.shape[0], prediction, int(objectIsInFrame), thisCoordinates[2], thisCoordinates[0], thisCoordinates[3], thisCoordinates[1]]
              if(prediction == "egg"):
                  thisColor = (0, 255, 0)
                  if(objectIsInFrame):
                      count = count + 1
              elif(prediction == "not egg"):
                  thisColor = (0, 0, 255)
              elif(prediction == "double egg"):
                  thisColor = (255, 0, 0)
                  if(objectIsInFrame):
                      count = count + 2
              cv2.rectangle(annotatedImage, (thisCoordinates[2], thisCoordinates[0]), (thisCoordinates[3], thisCoordinates[1]), thisColor, 2)


      annotatedImage = cv2.putText(annotatedImage, 'Egg count: ' + str(count), (100, 100), 1, 8, 1, 10)
      cv2.imwrite(str(os.path.splitext(os.path.basename(imagePath))[0]) + "_annotated.jpg", annotatedImage)
      inferences.to_csv(inferencesDirectory + 'inferencesInProgress.csv', index = False)
print("done")
inferences.to_csv(inferencesDirectory + 'inferences.csv', index = False)

1 out of 2224 images
22MIGS2HFL_0201_GS20-05-07_3_1315
3 out of 2224 images
22MIGS2HFL_0201_GS20-05-07_3_1317
4 out of 2224 images
22MIGS2HFL_0057_GS20-21-09_1_1318
8 out of 2224 images
22MIGS2HFL_0057_GS20-21-09_1_1322
16 out of 2224 images
22MIGS2HFL_0093_Covington(-)_1_1330
17 out of 2224 images
22MIGS2HFL_0093_Covington(-)_1_1331
18 out of 2224 images
22MIGS2HFL_0093_Covington(-)_1_1332
20 out of 2224 images
22MIGS2HFL_0292_PortoRico(-)_1_1334
21 out of 2224 images
22MIGS2HFL_0292_PortoRico(-)_1_1335
row corrected
24 out of 2224 images
22MIGS2HFL_0292_PortoRico(-)_1_1338
row corrected
25 out of 2224 images
22MIGS2HFL_0292_PortoRico(-)_1_1339
row corrected
26 out of 2224 images
22MIGS2HFL_0292_PortoRico(-)_1_1340
row corrected
27 out of 2224 images
22MIGS2HFL_0292_PortoRico(-)_1_1341
col corrected
row corrected
32 out of 2224 images
22MIGS2HFL_0257_GS20-26-04_3_1346
row corrected
34 out of 2224 images
22MIGS2HFL_0257_GS20-26-04_3_1348
43 out of 2224 images
22MIGS2HFL_0224_GS20-12-22